In [ ]:
path = ''

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd

basepath = Path(path)
files_in_basepath = basepath.iterdir()

dic = {}

for item in files_in_basepath:
  if item.is_file():
    
    name = item.name.split('_')
    
    tam = len(name)
    
    dataset = name[0]
    
    percent = name[tam-2]
    
    if name[1] == 'BoW':
      if name[2] == 'term-frequency-IDF':
        preprocessing = 'Bow-TFIDF'
      elif name[2] == 'term-frequency':
        preprocessing = 'Bow-TF'
      elif name[2] == 'binary':
        preprocessing = 'Bow-Binary'
    elif name[1] == 'DensityInformation':
      preprocessing = 'Density'
    else:
      preprocessing = name[1]
    

    df = pd.read_csv(path + item.name, sep=';')

    best_f1_mean = max(df['f1-score-mean'])

    best_f1_std = float(df[df['f1-score-mean'] == best_f1_mean]['f1-score-std'].iloc[0])

    best_pre_mean = float(df[df['f1-score-mean'] == best_f1_mean]['precision-mean'].iloc[0])

    best_pre_std = float(df[df['f1-score-mean'] == best_f1_mean]['precision-std'].iloc[0])

    brest_rev_mean = float(df[df['f1-score-mean'] == best_f1_mean]['recall-mean'].iloc[0])

    brest_rev_std = float(df[df['f1-score-mean'] == best_f1_mean]['recall-std'].iloc[0])

    best_aucroc_mean = max(df['auc_roc-mean'])

    best_aucroc_std = float(df[df['auc_roc-mean'] == best_aucroc_mean]['auc_roc-std'].iloc[0])

    best_acuracia_mean = max(df['accuracy-mean'])

    best_acuracia_std = float(df[df['accuracy-mean'] == best_acuracia_mean]['accuracy-std'].iloc[0])


    if dataset not in dic:
      dic[dataset] = {}
      dic[dataset][preprocessing] = pd.DataFrame(columns=['percent', 'precision-mean', 'precision-std',	'recall-mean', 'recall-std','f1-score-mean','f1-score-std',	'auc_roc-mean', 'auc_roc-std', 'accuracy-mean', 'accuracy-std'])
    elif preprocessing not in dic[dataset]:
      dic[dataset][preprocessing] = pd.DataFrame(columns=['percent', 'precision-mean', 'precision-std',	'recall-mean', 'recall-std','f1-score-mean','f1-score-std',	'auc_roc-mean', 'auc_roc-std', 'accuracy-mean', 'accuracy-std'])

    df_bests = dic[dataset][preprocessing]
    df_bests = df_bests.append({'percent' : percent.replace('.csv','_%'),
                                'precision-mean' : best_pre_mean, 
                                'precision-std' : best_pre_std, 
                                'recall-mean' : brest_rev_mean, 
                                'recall-std' : brest_rev_std, 
                                'f1-score-mean' : best_f1_mean, 
                                'f1-score-std' : best_f1_std, 
                                'auc_roc-mean' : best_aucroc_mean, 
                                'auc_roc-std' : best_aucroc_std, 
                                'accuracy-mean' : best_acuracia_mean,
                                'accuracy-std' : best_acuracia_std} , 
                               ignore_index=True)
    dic[dataset][preprocessing] = df_bests
    

In [ ]:
dic['TIT']['MAE']

,percent,precision-mean,precision-std,recall-mean,recall-std,f1-score-mean,f1-score-std,auc_roc-mean,auc_roc-std,accuracy-mean,accuracy-std
0,0.5,0.538551,0.016757,0.894787,0.039795,0.672029,0.019619,0.781578,0.033026,0.698062,0.031261
1,0.25,0.518612,0.014125,0.888134,0.036837,0.654248,0.009724,0.757022,0.027410,0.686359,0.028209
2,1,0.558289,0.018578,0.876994,0.021623,0.681989,0.015514,0.798101,0.021938,0.715333,0.016215
3,0.75,0.548510,0.019185,0.903121,0.019864,0.682293,0.017034,0.789187,0.025438,0.706666,0.020758


In [ ]:
def return_df(dataset, metric):
  df = pd.DataFrame(columns=['percent'] + list(dic[dataset].keys()))
  percents = ['0.25','0.5','0.75','1']
  df['percent'] = percents

  for prepro in dic[dataset].keys():
    df_aux = dic[dataset][prepro]
    values_list = []
    for percent in percents:
      values_list.append(df_aux[df_aux.percent == percent][metric].iloc[0])
    
    df[prepro] = values_list

  return df

In [ ]:
metric = 'accuracy-mean'

df_are = return_df('ARE', metric)
df_ten = return_df('TEN', metric)
df_tit = return_df('TIT', metric)

In [ ]:
df_are.to_csv('ARE-Acurracy.csv',sep=';',index=False)

In [ ]:
df_tit.to_csv('TIT-Acurracy.csv',sep=';',index=False)

In [ ]:
df_ten.to_csv('TEN-Acurracy.csv',sep=';',index=False)